In [2]:
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol
from pyspark import keyword_only
from pyspark.ml.param import Param, Params, TypeConverters
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable

class VisitsTransformer(Transformer, HasInputCol, HasOutputCol, DefaultParamsReadable, DefaultParamsWritable,):
    
    @keyword_only
    def __init__(self, inputCol="visits", outputCol="domains"):
        super(VisitsTransformer, self).__init__()
        if inputCol is not None:
            self.setInputCol(inputCol)
        if outputCol is not None:
            self.setOutputCol(outputCol)
    
#     def getVisitsJsonSchema(self, inDf):
#         inputCol = self.getInputCol()
#         return F.schema_of_json(inDf.select(inputCol).limit(1).toPandas()[inputCol].iloc[0])
    
    def _transform(self, dataset):
        from urllib.parse import urlparse
        from pyspark.sql.types import ArrayType, StringType
        extractDomainsUdf = F.udf(lambda urls: [urlparse(url).netloc.lower().replace('www.', '') 
                                        for url in urls], ArrayType(StringType()))
        visitsJsonSchema = ArrayType(StructType([StructField("timestamp", LongType()),
                                                StructField("url", StringType())]))
        inputCol = self.getInputCol()
        outputCol = self.getOutputCol()
        return dataset.withColumn(outputCol, extractDomainsUdf(F.from_json(inputCol, visitsJsonSchema).getField("url")))